# Part 4: Transparent Peer Review 분석

**SNU AI Psychology Workshop - February 2026**

---

## 학습 목표
1. **실제 피어리뷰** 데이터 수집하기 (Transparent Peer Review)
2. **Few-shot 리뷰 에이전트** 구현하기 (Human Review 예시 기반)
3. **Few-shot Reviewer vs agentic-paper-review** 비교 분석

---

## Transparent Peer Review란?

Nature Portfolio 저널들은 리뷰어 코멘트, 저자 응답, 편집자 결정문을 공개합니다.
- 논문 페이지 하단 → **"Peer review information"** 섹션
- **"Peer reviewer reports are available"** 클릭 → PDF 다운로드

### 리뷰 파일 공개 저널 목록

| 저널 | 분야 | 특징 |
|------|------|------|
| **[Communications Psychology](https://www.nature.com/commspsychol/)** | 심리학 | 모든 논문 필수 공개 |
| **[Nature Communications](https://www.nature.com/ncomms/)** | 다학제 | 2022년 이후 필수 공개 |
| **[Nature Human Behaviour](https://www.nature.com/nathumbehav/)** | 행동과학 | 옵션 제공 |
| **[Communications Biology](https://www.nature.com/commsbio/)** | 생물학/신경과학 | 모든 논문 필수 공개 |
| **[Communications Medicine](https://www.nature.com/commsmed/)** | 의학/건강 | 모든 논문 필수 공개 |
| **[Nature Biomedical Engineering](https://www.nature.com/natbiomedeng/)** | 의공학/AI 헬스 | 옵션 제공 |
| **[Nature](https://www.nature.com/nature/)** | 종합 과학 | 옵션 제공 |

> 💡 "필수 공개" 저널은 모든 논문에서 리뷰를 바로 다운로드할 수 있습니다.

---
## Part 1: Setup

In [1]:
# Cell 1: 환경 설정
import os
import sys

# Colab 환경 감지
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    WORKSHOP_DIR = "/content/drive/MyDrive/Hands-on-3/"
    os.makedirs(WORKSHOP_DIR, exist_ok=True)
    os.chdir(WORKSHOP_DIR)
    IN_COLAB = True
    print("🌐 Colab 환경에서 실행 중")
except ImportError:
    current_dir = os.getcwd()
    if current_dir.endswith('notebooks'):
        os.chdir('..')
    WORKSHOP_DIR = os.getcwd()
    IN_COLAB = False
    print("💻 로컬 환경에서 실행 중")

print(f"작업 폴더: {WORKSHOP_DIR}")

💻 로컬 환경에서 실행 중
작업 폴더: /Users/yejinlee/Documents/snuaipsych/yejin-lee/2026_SNUIPS_AI_Workshop/Hands-on-3


In [2]:
# Cell 2: API Key 로딩 및 의존성 설치
import os

# Colab에서 필요한 패키지 설치
if IN_COLAB:
    %pip install langchain-google-genai PyPDF2 -q
    print("✅ 패키지 설치 완료")

if IN_COLAB:
    try:
        from google.colab import userdata
        GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    except:
        GEMINI_API_KEY = None
else:
    from dotenv import load_dotenv
    load_dotenv(override=True)
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

if GEMINI_API_KEY and not GEMINI_API_KEY.startswith('your_'):
    os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY
    print("✅ Gemini API Key 로드 완료")
else:
    print("❌ Gemini API Key가 필요합니다")
    print("   발급: https://aistudio.google.com/apikey")

✅ Gemini API Key 로드 완료


---
## Part 2: 실제 리뷰 수집

---

### Transparent Peer Review 저널

| 저널 | 접속 | 예시 논문 |
|------|------|----------|
| **Communications Psychology** | [nature.com/commspsychol](https://www.nature.com/commspsychol/) | [예시](https://www.nature.com/articles/s44271-025-00291-w) |
| **Nature Communications** | [nature.com/ncomms](https://www.nature.com/ncomms/) | [예시](https://www.nature.com/articles/s41467-025-58072-2) |
| **OpenReview (ICLR)** | [openreview.net](https://openreview.net/group?id=ICLR.cc/2024/Conference#tab-accept-oral) | [예시](https://openreview.net/forum?id=xuY33XhEGR) |

---

### 다운로드 방법

**Nature 저널:**
1. 논문 페이지 **맨 아래** → **"Peer review information"**
2. **"Peer reviewer reports are available"** 클릭 → PDF 다운로드

**OpenReview:**
- 논문 페이지에서 **"Official Review"** 섹션 확인
- 복사/붙여넣기 또는 Chrome 인쇄로 저장

---

### 리뷰 저장

**저장 위치:** `input/human_reviews/`

**제공된 예시:** `example_review.pdf` (Climate change adaptation 논문 리뷰)

> ⚠️ **직접 1개 이상의 리뷰를 추가로 수집하세요!**

In [3]:
# Cell 3: 리뷰 폴더 생성 및 확인
import os
import urllib.request

REVIEW_DIR = os.path.join(WORKSHOP_DIR, "input", "human_reviews")
os.makedirs(REVIEW_DIR, exist_ok=True)

# Colab에서 example_review.pdf 없으면 GitHub에서 다운로드
example_path = os.path.join(REVIEW_DIR, "example_review.pdf")
if not os.path.exists(example_path):
    print("📥 example_review.pdf 다운로드 중...")
    url = "https://raw.githubusercontent.com/yoonlee78/2026_SNUIPS_AI_Workshop/master/Hands-on-3/input/human_reviews/example_review.pdf"
    try:
        urllib.request.urlretrieve(url, example_path)
        print("✅ example_review.pdf 다운로드 완료")
    except Exception as e:
        print(f"⚠️ 다운로드 실패: {e}")

# 기존 리뷰 파일 확인 (txt, pdf, md 모두 지원)
review_files = [f for f in os.listdir(REVIEW_DIR) if f.endswith(('.txt', '.pdf', '.md'))]

print(f"📁 리뷰 저장 위치: {REVIEW_DIR}")
print(f"\n📄 발견된 파일 ({len(review_files)}개):")
for f in review_files:
    if f == "example_review.pdf":
        print(f"   📄 {f} (예시 파일)")
    else:
        print(f"   📄 {f}")

# 최소 2개 이상 필요 (예시 파일 포함)
if len(review_files) >= 2:
    print(f"\n✅ 리뷰 파일 준비 완료!")
else:
    print(f"\n⚠️ 최소 2개 이상의 리뷰 파일이 필요합니다.")
    print("   example_review.pdf는 예시로 제공됩니다. 직접 1개 이상 추가하세요!")

📁 리뷰 저장 위치: /Users/yejinlee/Documents/snuaipsych/yejin-lee/2026_SNUIPS_AI_Workshop/Hands-on-3/input/human_reviews

📄 발견된 파일 (1개):
   📄 example_review.pdf (예시 파일)

⚠️ 최소 2개 이상의 리뷰 파일이 필요합니다.
   example_review.pdf는 예시로 제공됩니다. 직접 1개 이상 추가하세요!


In [4]:
# Cell 4: 리뷰 파일 확인
# Cell 3에서 정의된 review_files 사용, 없으면 다시 탐색
if 'review_files' not in dir() or not review_files:
    review_files = [f for f in os.listdir(REVIEW_DIR) if f.endswith(('.txt', '.pdf', '.md'))]

human_reviews = review_files  # 파일 목록 저장

print(f"✅ 총 {len(human_reviews)}개 리뷰 파일:")
for f in human_reviews:
    print(f"   📄 {f}")

✅ 총 1개 리뷰 파일:
   📄 example_review.pdf


---
## Part 3: Few-shot 리뷰 에이전트 구현

노트북 3에서는 **agentic-paper-review** (9노드 워크플로우)를 사용했습니다.

이번에는 **Transparent Peer Review를 Few-shot 예시로 활용**하는 에이전트를 구현합니다.

### Few-shot 접근법
```
[Human Review 예시] + [논문 텍스트] → [프롬프트] → [LLM] → [리뷰 + 점수]
```

실제 피어리뷰 예시를 프롬프트에 포함시켜 **리뷰 형식과 깊이**를 학습합니다.

In [5]:
# Cell 5: Few-shot 리뷰 에이전트 정의 (Transparent Peer Review 기반)
from langchain_google_genai import ChatGoogleGenerativeAI
import PyPDF2

# Gemma 3 12B: 무료 quota 최대 (30 RPM, 14,400 RPD)
# ⚠️ Gemma가 잘 안 되면 아래 모델로 변경 가능:
#    - "gemini-2.5-flash" (20 RPD, 더 높은 품질, quota 주의)
#    - "gemini-2.0-flash" (quota 제한 주의)
llm = ChatGoogleGenerativeAI(model="gemma-3-12b-it", temperature=0.3)

# 사용자 추가 리뷰 우선, 없으면 example_review.pdf 사용
user_reviews = [f for f in human_reviews if f != "example_review.pdf"]

if user_reviews:
    # 사용자가 추가한 리뷰 사용 (첫 번째 파일)
    review_file = user_reviews[0]
    print(f"📄 사용자 리뷰 사용: {review_file}")
else:
    # 사용자 리뷰 없으면 예시 파일 사용
    review_file = "example_review.pdf"
    print(f"📄 예시 리뷰 사용: {review_file} (사용자 리뷰 없음)")

review_path = os.path.join(REVIEW_DIR, review_file)

# 리뷰 텍스트 추출
example_review_text = ""
if os.path.exists(review_path):
    try:
        if review_file.endswith('.pdf'):
            with open(review_path, 'rb') as f:
                reader = PyPDF2.PdfReader(f)
                for page in reader.pages[:3]:  # 처음 3페이지만
                    example_review_text += page.extract_text() + "\n"
        else:  # .txt, .md
            with open(review_path, 'r', encoding='utf-8') as f:
                example_review_text = f.read()
        print(f"✅ 리뷰 로드 완료: {len(example_review_text)} 문자")
    except Exception as e:
        print(f"⚠️ 리뷰 읽기 실패: {e}")
else:
    print(f"⚠️ 리뷰 파일이 없습니다: {review_path}")

# Few-shot 프롬프트 (Human 리뷰 예시 포함) - 한글 출력
FEWSHOT_PROMPT = """당신은 학술 논문 리뷰어입니다. 모든 리뷰는 **한국어**로 작성하세요.

## 좋은 리뷰 예시 (Transparent Peer Review)
아래는 Nature 저널의 실제 피어리뷰 예시입니다. 이 형식과 깊이를 참고하세요:

---
{example_review}
---

## 리뷰할 논문:
{paper_content}

## 작성 지침:
위 예시처럼 구체적이고 건설적인 리뷰를 **한국어**로 작성하세요:

1. **요약 (Summary)**: 논문의 핵심 내용 2-3문장
2. **강점 (Strengths)**: 구체적인 강점 3개 이상
3. **약점 (Weaknesses)**: 건설적인 비판 3개 이상  
4. **세부 코멘트**: 섹션별 구체적 피드백
5. **점수**: Soundness, Presentation, Contribution (1-4점), Overall (1-10점)
6. **최종 결정**: Accept / Weak Accept / Weak Reject / Reject

**중요: 반드시 한국어로 작성하세요.**
"""

def fewshot_review_agent(paper_content):
    """Few-shot 리뷰 에이전트 (Transparent Peer Review 기반)"""
    prompt = FEWSHOT_PROMPT.format(
        example_review=example_review_text[:5000] if example_review_text else "[예시 없음]",
        paper_content=paper_content[:12000]
    )
    
    response = llm.invoke(prompt)
    
    return {
        'review': response.content,
        'model': 'gemma-3-12b-it',
        'method': 'few-shot (transparent peer review)',
        'review_source': review_file
    }

print("✅ fewshot_review_agent() 정의 완료")
print("   모델: gemma-3-12b-it (30 RPM, 14,400 RPD)")
print("   방식: Few-shot (Transparent Peer Review 기반)")
print("   출력 언어: 한국어")

/Users/yejinlee/Documents/snuaipsych/yejin-lee/2026_SNUIPS_AI_Workshop/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


📄 예시 리뷰 사용: example_review.pdf (사용자 리뷰 없음)
✅ 리뷰 로드 완료: 12430 문자
✅ fewshot_review_agent() 정의 완료
   모델: gemma-3-12b-it (30 RPM, 14,400 RPD)
   방식: Few-shot (Transparent Peer Review 기반)
   출력 언어: 한국어


In [6]:
# Cell 6: 논문 로딩 및 Few-shot 리뷰 생성
import urllib.request

# ✏️ 본인 논문 경로 입력 (비워두면 자동 탐색)
MY_PAPER_PATH = ""  # 예: "input/my_draft.md"

# ─────────────────────────────────────────
PAPER_PATH = None
paper_content = None

# Colab: sample_manuscript.md 없으면 GitHub에서 다운로드
INPUT_DIR = os.path.join(WORKSHOP_DIR, "input")
os.makedirs(INPUT_DIR, exist_ok=True)
SAMPLE_PATH = os.path.join(INPUT_DIR, "sample_manuscript.md")

if not os.path.exists(SAMPLE_PATH):
    print("📥 sample_manuscript.md 다운로드 중...")
    url = "https://raw.githubusercontent.com/yoonlee78/2026_SNUIPS_AI_Workshop/master/Hands-on-3/input/sample_manuscript.md"
    try:
        urllib.request.urlretrieve(url, SAMPLE_PATH)
        print("✅ sample_manuscript.md 다운로드 완료")
    except Exception as e:
        print(f"⚠️ 다운로드 실패: {e}")

# 1순위: 직접 지정한 경로
if MY_PAPER_PATH and os.path.exists(MY_PAPER_PATH):
    PAPER_PATH = MY_PAPER_PATH
    with open(MY_PAPER_PATH, 'r', encoding='utf-8') as f:
        paper_content = f.read()
    print(f"✅ 본인 논문 로드: {MY_PAPER_PATH}")

# 2순위: my_manuscript.md (노트북 2.5에서 생성)
if PAPER_PATH is None:
    MY_MANUSCRIPT = os.path.join(WORKSHOP_DIR, "input", "my_manuscript.md")
    if os.path.exists(MY_MANUSCRIPT):
        PAPER_PATH = MY_MANUSCRIPT
        with open(MY_MANUSCRIPT, 'r', encoding='utf-8') as f:
            paper_content = f.read()
        print(f"✅ 내 논문 사용: my_manuscript.md")

# 3순위: sample_manuscript.md (기본 샘플)
if PAPER_PATH is None:
    if os.path.exists(SAMPLE_PATH):
        PAPER_PATH = SAMPLE_PATH
        with open(SAMPLE_PATH, 'r', encoding='utf-8') as f:
            paper_content = f.read()
        print(f"📄 샘플 논문 사용: sample_manuscript.md")
        print("")
        print("="*50)
        print("💡 본인 논문으로 실습하려면:")
        print("   1. 노트북 2.5 실행 → my_manuscript.md 생성")
        print("   2. 또는 MY_PAPER_PATH에 파일 경로 입력")
        print("="*50)

if paper_content:
    print(f"\n📄 논문: {PAPER_PATH}")
    print(f"   텍스트 길이: {len(paper_content)} 문자")
    print("\n🔄 Few-shot 리뷰 생성 중...")
    
    fewshot_review = fewshot_review_agent(paper_content)
    
    print("\n" + "="*60)
    print("🤖 Few-shot Reviewer 결과")
    print("   (Transparent Peer Review 예시 기반)")
    print("="*60)
    print(fewshot_review['review'][:2000])
    if len(fewshot_review['review']) > 2000:
        print(f"\n... ({len(fewshot_review['review'])}자 중 2000자만 표시)")
else:
    print("❌ 논문 파일이 없습니다.")
    print("   노트북 2.5를 먼저 실행하세요.")

✅ 내 논문 사용: my_manuscript.md

📄 논문: /Users/yejinlee/Documents/snuaipsych/yejin-lee/2026_SNUIPS_AI_Workshop/Hands-on-3/input/my_manuscript.md
   텍스트 길이: 12961 문자

🔄 Few-shot 리뷰 생성 중...

🤖 Few-shot Reviewer 결과
   (Transparent Peer Review 예시 기반)
## 논문 리뷰: The Longitudinal Association Between Social Media Usage, Geographic Location, and Trait Anxiety in Adolescents: An Evolutionary Perspective

**1. 요약 (Summary)**

본 논문은 캘리포니아 주 내 세 지역의 청소년을 대상으로 소셜 미디어 사용, 거주 지역(기후 변화 영향 지역과의 근접성), 그리고 성격 불안의 장기적인 연관성을 조사한 연구입니다. 연구 결과, 소셜 미디어 사용과 성격 불안 사이에 유의미한 양의 상관관계가 있으며, 기후 변화 영향 지역에 거주하는 청소년의 불안 수준이 더 높다는 것을 확인했습니다. 진화론적 관점에서 이러한 결과를 해석하고, 청소년의 정신 건강에 대한 시사점을 제시하고 있습니다.

**2. 강점 (Strengths)**

*   **중요한 연구 주제:** 청소년의 정신 건강, 소셜 미디어 사용, 그리고 기후 변화라는 현대 사회의 중요한 문제들을 다루고 있어 사회적 의미가 큽니다. 특히, 세 가지 요인의 복합적인 연관성을 탐구한 점은 주목할 만합니다.
*   **장기적인 연구 설계:** 2년 동안 데이터를 수집하여 시간의 흐름에 따른 변화를 파악하려는 시도는 연구의 신뢰도를 높이는 데 기여합니다.
*   **다양한 변수 고려:** 소셜 미디어 사용 시간, 거주 지역의 기후 변화 영향 정도, 그리고 성격 불안을 측정하여 다각적인 분석을 시도했습니다.
*   **진화론적 관점

---
## Part 4: Few-shot Reviewer vs agentic-paper-review 비교

**핵심 비교:**
- **Few-shot Reviewer** (노트북 4): Transparent Peer Review 예시 기반 단일 프롬프트
- **agentic-paper-review** (노트북 3): 9노드 워크플로우, 웹검색, 리플렉션

수집한 Human 리뷰는 Few-shot의 예시로 활용됩니다.

In [ ]:
# Cell 7: 노트북 3 결과 로드 (Agentic Agent)
import json
import os
import urllib.request

notebook3_dir = os.path.join(WORKSHOP_DIR, "outputs", "3_agent_result")
notebook3_path = os.path.join(notebook3_dir, "review.json")

# Colab: review.json 없으면 GitHub에서 샘플 다운로드
if not os.path.exists(notebook3_path):
    os.makedirs(notebook3_dir, exist_ok=True)
    print("📥 샘플 review.json 다운로드 중...")
    url = "https://raw.githubusercontent.com/yoonlee78/2026_SNUIPS_AI_Workshop/master/Hands-on-3/outputs/3_agent_result/sample_review.json"
    try:
        urllib.request.urlretrieve(url, notebook3_path)
        print("✅ 샘플 review.json 다운로드 완료")
        print("   (노트북 3 실행 시 본인 결과로 덮어씌워집니다)")
    except Exception as e:
        print(f"⚠️ 다운로드 실패: {e}")

# 디버깅 정보
print(f"\n📂 WORKSHOP_DIR: {WORKSHOP_DIR}")
print(f"📄 찾는 파일: {notebook3_path}")
print(f"   파일 존재: {os.path.exists(notebook3_path)}")

# 폴더 내용 확인
if os.path.exists(notebook3_dir):
    files = os.listdir(notebook3_dir)
    print(f"\n📁 3_agent_result 폴더 내용: {files}")

print("")

notebook3_review = None
if os.path.exists(notebook3_path):
    with open(notebook3_path, 'r', encoding='utf-8') as f:
        notebook3_review = json.load(f)
    
    if notebook3_review.get('status') == 'complete':
        print("✅ 노트북 3 결과 로드 완료 (Agentic Agent)")
        scores = notebook3_review.get('scores', {})
        print(f"   최종 점수: {scores.get('final_score_display', 'N/A')}")
    else:
        print("⚠️ 노트북 3 결과가 완료 상태가 아닙니다")
        print(f"   상태: {notebook3_review.get('status', 'unknown')}")
        notebook3_review = None
else:
    print("⚠️ 노트북 3 결과 파일이 없습니다")
    print("   → 노트북 3 (Cell 8까지) 실행 후 다시 시도하세요")

In [13]:
# Cell 8: Few-shot Reviewer vs agentic-paper-review 비교 (HTML)
from IPython.display import HTML, display
from datetime import datetime
import re

# 출력 폴더 및 timestamp 설정
output_dir = os.path.join(WORKSHOP_DIR, "outputs", "4_comparison_result")
os.makedirs(output_dir, exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

def markdown_to_html(text):
    """간단한 마크다운 → HTML 변환"""
    # **bold** → <strong>bold</strong>
    text = re.sub(r'\*\*(.+?)\*\*', r'<strong>\1</strong>', text)
    # *italic* → <em>italic</em>
    text = re.sub(r'\*(.+?)\*', r'<em>\1</em>', text)
    # ## heading → <h3>heading</h3>
    text = re.sub(r'^## (.+)$', r'<h3>\1</h3>', text, flags=re.MULTILINE)
    # ### heading → <h4>heading</h4>
    text = re.sub(r'^### (.+)$', r'<h4>\1</h4>', text, flags=re.MULTILINE)
    # - list item → <li>list item</li>
    text = re.sub(r'^- (.+)$', r'<li>\1</li>', text, flags=re.MULTILINE)
    # 연속된 <li>를 <ul>로 감싸기
    text = re.sub(r'((?:<li>.*</li>\n?)+)', r'<ul>\1</ul>', text)
    # 줄바꿈 → <br>
    text = text.replace('\n\n', '</p><p>').replace('\n', '<br>')
    text = f'<p>{text}</p>'
    return text

if 'fewshot_review' in dir() and notebook3_review:
    
    # 리뷰 내용 준비 (마크다운 → HTML 변환)
    fewshot_text = fewshot_review['review']
    fewshot_html = markdown_to_html(fewshot_text)
    
    agentic_text = notebook3_review.get('review', '')
    agentic_html = markdown_to_html(agentic_text)
    agentic_score = notebook3_review.get('scores', {}).get('final_score_display', 'N/A')
    
    # 관련 연구 목록
    related = notebook3_review.get('related_works', [])
    related_html = ""
    if related:
        related_html = "<h4>📚 관련 연구</h4><ul>"
        for w in related[:3]:
            related_html += f"<li><a href='{w.get('url', '#')}'>{w['title'][:60]}...</a></li>"
        related_html += "</ul>"
    
    # EMPHL 스타일 HTML
    full_html = f"""<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Few-shot vs Agentic 리뷰 비교</title>
    <link href="https://fonts.googleapis.com/css2?family=Noto+Sans+KR:wght@300;400;500;700&display=swap" rel="stylesheet">
    <style>
        body {{
            font-family: 'Noto Sans KR', -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
            line-height: 1.8;
            max-width: 1400px;
            margin: 0 auto;
            padding: 40px 20px;
            color: #333;
            background-color: #f9f9f9;
        }}

        h1 {{
            color: #2c3e50;
            border-bottom: 3px solid #3498db;
            padding-bottom: 15px;
            margin-top: 20px;
            margin-bottom: 30px;
            font-size: 2em;
            text-align: center;
        }}

        h2 {{
            color: #34495e;
            margin-top: 35px;
            margin-bottom: 20px;
            padding-bottom: 10px;
            border-bottom: 2px solid #e0e0e0;
            font-size: 1.5em;
        }}

        h3 {{
            color: #555;
            margin-top: 25px;
            margin-bottom: 15px;
            font-size: 1.2em;
        }}

        h4 {{
            color: #666;
            margin-top: 20px;
            margin-bottom: 12px;
            font-size: 1.1em;
        }}

        .timestamp {{
            text-align: center;
            color: #888;
            margin-bottom: 30px;
        }}

        table {{
            border-collapse: collapse;
            width: 100%;
            margin: 20px 0;
            background-color: white;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }}

        th {{
            background-color: #3498db;
            color: white;
            padding: 12px;
            text-align: left;
            font-weight: 500;
        }}

        td {{
            padding: 12px;
            border-bottom: 1px solid #e0e0e0;
            vertical-align: top;
        }}

        tr:hover {{
            background-color: #f5f5f5;
        }}

        .pros-cons {{
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 20px;
            margin: 20px 0;
        }}

        .pros, .cons {{
            padding: 20px;
            border-radius: 8px;
            max-height: 70vh;
            overflow-y: auto;
        }}

        .pros {{
            background-color: #e8f5e9;
            border-left: 4px solid #4caf50;
        }}

        .cons {{
            background-color: #e3f2fd;
            border-left: 4px solid #2196f3;
        }}

        .pros h3 {{
            color: #4caf50;
            margin-top: 0;
        }}

        .cons h3 {{
            color: #2196f3;
            margin-top: 0;
        }}

        .meta-box {{
            background-color: #f8f8f8;
            border: 1px solid #e0e0e0;
            border-radius: 5px;
            padding: 12px 15px;
            margin-bottom: 15px;
            font-size: 0.95em;
        }}

        .score {{
            display: inline-block;
            background-color: #2196f3;
            color: white;
            padding: 4px 12px;
            border-radius: 4px;
            font-weight: 500;
        }}

        .review-content {{
            font-size: 14px;
            line-height: 1.9;
            color: #333;
        }}
        
        .review-content p {{
            margin: 10px 0;
        }}
        
        .review-content h3 {{
            color: #2c3e50;
            border-bottom: 1px solid #ddd;
            padding-bottom: 5px;
        }}
        
        .review-content h4 {{
            color: #34495e;
        }}
        
        .review-content ul {{
            margin: 10px 0;
            padding-left: 20px;
        }}
        
        .review-content li {{
            margin: 5px 0;
        }}
        
        .review-content strong {{
            color: #2c3e50;
        }}

        ul {{
            margin: 15px 0;
            padding-left: 25px;
        }}

        li {{
            margin: 8px 0;
            line-height: 1.7;
        }}

        a {{
            color: #3498db;
            text-decoration: none;
        }}

        a:hover {{
            text-decoration: underline;
        }}

        hr {{
            border: none;
            border-top: 2px solid #e0e0e0;
            margin: 30px 0;
        }}

        @media print {{
            .pros, .cons {{
                max-height: none;
                overflow: visible;
            }}
            
            body {{
                background-color: white;
            }}
        }}

        @media (max-width: 900px) {{
            .pros-cons {{
                grid-template-columns: 1fr;
            }}
        }}
    </style>
</head>
<body>

<h1>Few-shot vs Agentic 리뷰 비교</h1>
<p class="timestamp">생성: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}</p>

<h2>방식 비교</h2>
<table>
<thead>
<tr>
<th>항목</th>
<th>Few-shot Reviewer</th>
<th>agentic-paper-review</th>
</tr>
</thead>
<tbody>
<tr>
<td><strong>기반</strong></td>
<td>Transparent Peer Review 예시</td>
<td>9노드 워크플로우</td>
</tr>
<tr>
<td><strong>모델</strong></td>
<td>gemma-3-12b-it</td>
<td>gemini-2.5-flash</td>
</tr>
<tr>
<td><strong>웹검색</strong></td>
<td>없음</td>
<td>있음 (관련 논문 탐색)</td>
</tr>
<tr>
<td><strong>리플렉션</strong></td>
<td>없음</td>
<td>있음 (자기 검토)</td>
</tr>
<tr>
<td><strong>Few-shot 예시</strong></td>
<td>있음 (Human Review)</td>
<td>없음</td>
</tr>
<tr>
<td><strong>최종 점수</strong></td>
<td>-</td>
<td><span class="score">{agentic_score}</span></td>
</tr>
</tbody>
</table>

<hr>

<h2>리뷰 내용 비교</h2>

<div class="pros-cons">
    <div class="pros">
        <h3>🟢 Few-shot Reviewer</h3>
        <div class="meta-box">
            <strong>모델:</strong> gemma-3-12b-it<br>
            <strong>방식:</strong> Transparent Peer Review 예시 기반 (단일 프롬프트)
        </div>
        <div class="review-content">{fewshot_html}</div>
    </div>
    <div class="cons">
        <h3>🔵 agentic-paper-review</h3>
        <div class="meta-box">
            <strong>방식:</strong> 9노드 워크플로우 (웹검색 + 리플렉션)<br>
            <strong>점수:</strong> <span class="score">{agentic_score}</span>
        </div>
        <div class="review-content">{agentic_html}</div>
        {related_html}
    </div>
</div>

</body>
</html>"""
    
    # HTML 파일 저장
    html_path = os.path.join(output_dir, f"comparison_{timestamp}.html")
    with open(html_path, 'w', encoding='utf-8') as f:
        f.write(full_html)
    
    latest_html = os.path.join(output_dir, "comparison.html")
    with open(latest_html, 'w', encoding='utf-8') as f:
        f.write(full_html)
    
    # 노트북 내 미리보기 (간소화)
    preview_html = f"""
    <style>
        .preview-container {{ display: flex; gap: 15px; margin: 15px 0; }}
        .preview-panel {{ flex: 1; border: 1px solid #e0e0e0; border-radius: 8px; padding: 15px; 
                         max-height: 400px; overflow-y: auto; background: white; font-family: 'Noto Sans KR', sans-serif; }}
        .preview-panel h4 {{ margin-top: 0; padding-bottom: 8px; border-bottom: 2px solid; }}
        .fewshot h4 {{ border-color: #4caf50; color: #4caf50; }}
        .agentic h4 {{ border-color: #2196f3; color: #2196f3; }}
        .preview-content {{ font-size: 12px; line-height: 1.6; }}
        .preview-content strong {{ color: #2c3e50; }}
    </style>
    <h3>📊 리뷰 비교 미리보기</h3>
    <div class="preview-container">
        <div class="preview-panel fewshot">
            <h4>🟢 Few-shot Reviewer</h4>
            <div class="preview-content">{markdown_to_html(fewshot_text[:2000])}{'...' if len(fewshot_text) > 2000 else ''}</div>
        </div>
        <div class="preview-panel agentic">
            <h4>🔵 agentic-paper-review ({agentic_score})</h4>
            <div class="preview-content">{markdown_to_html(agentic_text[:2000])}{'...' if len(agentic_text) > 2000 else ''}</div>
        </div>
    </div>
    """
    display(HTML(preview_html))
    
    print(f"\n💾 HTML 비교 파일 저장:")
    print(f"   📄 {output_dir}/comparison_{timestamp}.html")
    print(f"   📄 {output_dir}/comparison.html (최신)")
    print(f"\n🌐 브라우저에서 전체 보기: file://{os.path.abspath(latest_html)}")
    
else:
    print("⚠️ 비교를 위해 두 가지가 모두 필요합니다:")
    if 'fewshot_review' not in dir():
        print("   - Few-shot Reviewer 리뷰: Cell 6 실행 필요")
    if not notebook3_review:
        print("   - agentic-paper-review 리뷰: 노트북 3 실행 필요")


💾 HTML 비교 파일 저장:
   📄 /Users/yejinlee/Documents/snuaipsych/yejin-lee/2026_SNUIPS_AI_Workshop/Hands-on-3/outputs/4_comparison_result/comparison_20260203_001943.html
   📄 /Users/yejinlee/Documents/snuaipsych/yejin-lee/2026_SNUIPS_AI_Workshop/Hands-on-3/outputs/4_comparison_result/comparison.html (최신)

🌐 브라우저에서 전체 보기: file:///Users/yejinlee/Documents/snuaipsych/yejin-lee/2026_SNUIPS_AI_Workshop/Hands-on-3/outputs/4_comparison_result/comparison.html


---
## Part 5: 결과 저장 및 제출

### 결과 제출 방법

1. **Cell 9 실행** → JSON, Markdown, HTML 저장
2. **Cell 10에서 이름 입력**
3. **셀 실행 → 파란 버튼 클릭**
4. **폼에서 의견 작성 + HTML 파일 업로드 → 제출**

> 이름과 논문 유형은 자동 입력됩니다!

### 링크

| 용도 | 링크 |
|------|------|
| **직접 제출** (자동 입력 안 될 때) | [Google Forms](https://docs.google.com/forms/d/e/1FAIpQLSeYrzmXSEmoddzInY5j5xagy4cfa-MwolZYZYvm6_B7gXvnNQ/viewform) |

### 제출 항목

| 항목 | 자동 입력 |
|------|----------|
| 이름 | ✅ |
| 사용한 논문 | ✅ |
| 어느 리뷰가 더 유용했나요? | ❌ (직접 선택) |
| 어떤 점에서 더 유용했나요? | ❌ (직접 작성) |
| 어떤 방식을 사용하겠습니까? | ❌ (직접 선택) |
| 두 방식을 어떻게 결합하면 좋을까요? | ❌ (직접 작성) |
| HTML 비교 파일 업로드 | ❌ (직접 업로드) |

In [14]:
# Cell 9: 결과 저장 (.md + .json, timestamp 포함)
import json
from datetime import datetime

output_dir = os.path.join(WORKSHOP_DIR, "outputs", "4_comparison_result")
os.makedirs(output_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# 1. Few-shot 리뷰 Markdown 저장
if 'fewshot_review' in dir() and fewshot_review:
    md_content = f"""# Few-shot Reviewer 결과

**생성 시간:** {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
**모델:** {fewshot_review.get('model', 'gemini-2.5-flash')}
**방식:** {fewshot_review.get('method', 'few-shot (transparent peer review)')}

---

{fewshot_review['review']}
"""
    md_path = os.path.join(output_dir, f"fewshot_review_{timestamp}.md")
    with open(md_path, 'w', encoding='utf-8') as f:
        f.write(md_content)
    print(f"📄 Markdown 저장: fewshot_review_{timestamp}.md")

# 2. 비교 결과 JSON 저장
comparison_result = {
    'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'human_reviews_count': len(human_reviews) if human_reviews else 0,
    'human_reviews': human_reviews if human_reviews else [],
    'fewshot_review': fewshot_review if 'fewshot_review' in dir() else None,
    'agentic_review_path': notebook3_path if notebook3_review else None,
    'agentic_review_score': notebook3_review.get('scores', {}).get('final_score_display') if notebook3_review else None
}

json_path = os.path.join(output_dir, f"comparison_{timestamp}.json")
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(comparison_result, f, ensure_ascii=False, indent=2)
print(f"📋 JSON 저장: comparison_{timestamp}.json")

# 3. 최신 결과 링크 (timestamp 없는 버전)
latest_md = os.path.join(output_dir, "fewshot_review.md")
latest_json = os.path.join(output_dir, "comparison.json")

if 'fewshot_review' in dir() and fewshot_review:
    with open(latest_md, 'w', encoding='utf-8') as f:
        f.write(md_content)
        
with open(latest_json, 'w', encoding='utf-8') as f:
    json.dump(comparison_result, f, ensure_ascii=False, indent=2)

print(f"\n💾 결과 저장 완료: {output_dir}")
print(f"   - fewshot_review_{timestamp}.md")
print(f"   - comparison_{timestamp}.json")
print(f"   - fewshot_review.md (최신)")
print(f"   - comparison.json (최신)")

📄 Markdown 저장: fewshot_review_20260203_002036.md
📋 JSON 저장: comparison_20260203_002036.json

💾 결과 저장 완료: /Users/yejinlee/Documents/snuaipsych/yejin-lee/2026_SNUIPS_AI_Workshop/Hands-on-3/outputs/4_comparison_result
   - fewshot_review_20260203_002036.md
   - comparison_20260203_002036.json
   - fewshot_review.md (최신)
   - comparison.json (최신)


In [15]:
# Cell 10: 결과 제출
# ============================================================
# 이름만 입력하세요! 나머지는 자동으로 채워집니다.
# ============================================================

PARTICIPANT_NAME = "이예진_final"  # <- 본인 이름

# ============================================================
# 실험 결과 자동 수집
# ============================================================
import urllib.parse
import json
from datetime import datetime
from IPython.display import display, HTML

if not PARTICIPANT_NAME:
    print("⚠️  PARTICIPANT_NAME을 입력하세요!")
elif 'fewshot_review' not in dir() or not fewshot_review:
    print("⚠️  Cell 6을 먼저 실행하세요! (Few-shot 리뷰 생성)")
else:
    # 사용한 논문 유형
    paper_type = "샘플논문" if not MY_PAPER_PATH or "sample_manuscript" in str(PAPER_PATH) else "본인 논문"
    
    # Pre-filled Google Form URL 생성
    FORM_BASE = "https://docs.google.com/forms/d/e/1FAIpQLSeYrzmXSEmoddzInY5j5xagy4cfa-MwolZYZYvm6_B7gXvnNQ/viewform"
    
    params = {
        "usp": "pp_url",
        "entry.2147291049": PARTICIPANT_NAME,  # 이름
        "entry.421813704": paper_type,         # 사용한 논문
        # 나머지는 폼에서 직접 작성
        # entry.571419321: 어느 리뷰가 더 유용했나요
        # entry.939795068: 어떤 점에서 더 유용했나요
        # (추가 entry ID는 폼에서 확인 필요)
    }
    
    form_url = f"{FORM_BASE}?{urllib.parse.urlencode(params)}"
    
    # 결과 미리보기
    print("=" * 70)
    print("📋 제출 내용 미리보기")
    print("=" * 70)
    print(f"\n👤 이름: {PARTICIPANT_NAME}")
    print(f"📄 논문: {paper_type}")
    print(f"\n🟢 Few-shot Reviewer: 생성 완료 ({len(fewshot_review.get('review', ''))} 문자)")
    
    if notebook3_review:
        agentic_score = notebook3_review.get('scores', {}).get('final_score_display', 'N/A')
        print(f"🔵 agentic-paper-review: {agentic_score}")
    else:
        print(f"🔵 agentic-paper-review: 결과 없음 (노트북 3 실행 필요)")
    
    # 클릭 가능한 링크 출력
    print("\n" + "=" * 70)
    print("🔗 아래 버튼 클릭 → 의견 작성 → HTML 파일 업로드 → 제출")
    print("=" * 70)
    
    display(HTML(f'''
    <a href="{form_url}" target="_blank" 
       style="display:inline-block; font-size:16px; padding:12px 24px; 
              background:#4285f4; color:white; text-decoration:none; 
              border-radius:5px; margin:10px 0;">
       📝 결과 제출하기 (클릭)
    </a>
    <p style="color:#666; font-size:12px;">※ 이름과 논문 유형은 자동 입력됩니다.</p>
    <p style="color:#666; font-size:12px;">※ 업로드할 파일: <code>outputs/4_comparison_result/comparison.html</code></p>
    '''))
    
    # HTML 파일 경로 안내
    html_path = os.path.join(output_dir, "comparison.html")
    if os.path.exists(html_path):
        print(f"\n📎 업로드할 HTML 파일:")
        print(f"   {html_path}")
    
    # 로컬 백업 저장
    save_data = {
        'participant': PARTICIPANT_NAME,
        'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'paper_type': paper_type,
        'fewshot_review_length': len(fewshot_review.get('review', '')),
        'agentic_score': notebook3_review.get('scores', {}).get('final_score_display') if notebook3_review else None,
        'fewshot_review_preview': fewshot_review.get('review', '')[:500]  # 처음 500자만
    }
    
    save_folder = os.path.join(WORKSHOP_DIR, "outputs", "4_comparison_result")
    os.makedirs(save_folder, exist_ok=True)
    filename = f"{PARTICIPANT_NAME}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    filepath = os.path.join(save_folder, filename)
    
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(save_data, f, ensure_ascii=False, indent=2)
    
    print(f"\n✓ 로컬 백업: {filepath}")

📋 제출 내용 미리보기

👤 이름: 이예진_final
📄 논문: 샘플논문

🟢 Few-shot Reviewer: 생성 완료 (2669 문자)
🔵 agentic-paper-review: 5.1/10

🔗 아래 버튼 클릭 → 의견 작성 → HTML 파일 업로드 → 제출



📎 업로드할 HTML 파일:
   /Users/yejinlee/Documents/snuaipsych/yejin-lee/2026_SNUIPS_AI_Workshop/Hands-on-3/outputs/4_comparison_result/comparison.html

✓ 로컬 백업: /Users/yejinlee/Documents/snuaipsych/yejin-lee/2026_SNUIPS_AI_Workshop/Hands-on-3/outputs/4_comparison_result/이예진_final_20260203_002226.json


---
## 정리

### 이번 노트북에서 배운 것

1. **Transparent Peer Review** - Nature Portfolio, OpenReview에서 공개 리뷰 수집
2. **Few-shot 리뷰 에이전트 구현** - Human Review 예시 기반 단일 프롬프트
3. **리뷰 방식 비교** - Few-shot Reviewer vs agentic-paper-review

### 리뷰 에이전트 비교

| 방식 | 장점 | 단점 |
|------|------|------|
| **Few-shot Reviewer** | Human 리뷰 스타일 학습, 빠름, 비용 저렴 | 예시 품질에 의존, 웹검색 없음 |
| **agentic-paper-review** | 웹검색, 리플렉션, 관련 논문 추천 | 복잡, API 비용 높음 |

### 핵심 차이점

| 항목 | Few-shot Reviewer | agentic-paper-review |
|------|-------------------|---------------------|
| **프롬프트** | Transparent Peer Review 예시 포함 | 구조화된 멀티 노드 |
| **노드 수** | 1 | 9 |
| **웹검색** | ❌ | ✅ |
| **리플렉션** | ❌ | ✅ |

### 공개 리뷰 소스

| 소스 | 분야 | 특징 |
|------|------|------|
| [Communications Psychology](https://www.nature.com/commspsychol/) | 심리학 | 모든 논문 리뷰 공개 |
| [Nature Communications](https://www.nature.com/ncomms/) | 다학제 | 2022년 이후 전체 공개 |
| [OpenReview](https://openreview.net/) | AI/ML | ICLR, NeurIPS 등 |